In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
en = cptac.Endometrial()

In [63]:
mut = en.get_genotype_all_vars("PIK3CA", mutation_hotspot=['E542K', 'E545K', 'H1047R'])
prot = en.get_proteomics(tissue_type="tumor")

joined = mut.join(prot)

en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]

wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

endo_hotspot = en_hotspot_df.append(wildtype)

prot_and_mutations = endo_hotspot.drop(columns = ["Mutation_Status", "Location"])
prot_and_mutations

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


Name,Mutation,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,-1.1800,-0.86300,-0.8020,0.22200,0.2560,0.66500,1.28000,-0.33900,0.41200,...,-0.087700,NaN,0.02290,0.1090,NaN,-0.3320,-0.43300,-1.02000,-0.12300,-0.08590
C3L-00032,Missense_Mutation,-0.5280,-1.32000,0.4350,NaN,-0.2400,1.04000,-0.02130,-0.04790,0.41900,...,0.001120,-0.14500,0.01050,-0.1160,NaN,0.1510,-0.07400,-0.54000,0.32000,-0.41900
C3L-00362,Missense_Mutation,-0.9240,-0.44500,1.5700,-0.67800,0.1730,0.43600,0.22700,-0.00248,0.47900,...,-0.049600,NaN,0.10800,-0.1530,NaN,0.1090,-0.33600,-0.82200,-0.03380,0.12100
C3L-00601,Missense_Mutation,-0.4540,-0.24200,NaN,NaN,0.2580,0.21900,-0.24900,-1.33000,0.20400,...,-0.158000,NaN,0.66700,1.3000,0.4410,0.1300,-0.06590,-0.92300,-0.16300,0.11200
C3L-00605,Missense_Mutation,-0.2400,0.59400,3.4000,0.15400,0.0932,0.28300,-0.07890,-0.61100,0.37000,...,-0.051100,-0.01360,0.21400,0.6830,0.4330,-0.2080,-0.34700,-0.91100,0.06920,-0.23200
C3L-00921,Missense_Mutation,-1.1900,-1.19000,3.6600,1.27000,-0.1050,-0.00210,0.08130,-0.69100,-0.29500,...,0.097000,0.25400,0.15100,0.2850,-0.2390,-0.0681,-0.21800,-0.08730,-0.12700,0.37400
C3L-00947,Missense_Mutation,-0.4850,0.91700,0.3040,-0.00675,0.1780,-0.42300,0.22400,-0.24700,0.07870,...,0.042900,-0.21200,0.16900,0.5210,-0.1470,0.3540,0.17000,-1.01000,-0.11000,0.13100
C3N-00323,Missense_Mutation,-0.8170,-1.13000,NaN,NaN,0.1020,0.37100,0.64400,0.28100,0.11400,...,-0.022200,NaN,-0.27900,0.4230,NaN,0.2940,-0.13000,-0.10600,-0.29900,0.38500
C3N-00324,Missense_Mutation,-0.4490,-0.44000,1.1500,NaN,-0.6450,0.31600,0.08090,-0.47000,0.80400,...,0.044900,0.04520,-0.15200,-0.1970,0.2950,-0.1880,-0.26500,0.25000,-0.11300,-0.30200


In [56]:
label_values = prot_and_mutations["Mutation"].unique()

'''Partition dataframe into two sets, one for each of the two unique values from the label column'''
partition1 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == label_values[0]]
partition2 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == label_values[1]]

'''If no comparison columns specified, use all columns except the specified labed column'''
comparison_columns = list(prot_and_mutations.columns)
comparison_columns.remove("Mutation")

number_of_comparisons = len(comparison_columns)

'''Store comparisons and p-values in two arrays'''
comparisons = []
pvals = []

'''Loop through each comparison column, perform the t-test, and record the p-val'''
times_through = 0
for column in comparison_columns:  
    times_through += 1
    if len(partition1[column].dropna(axis=0)) <= 1:
#         comparison_columns.remove(column)
        continue
    elif len(partition2[column].dropna(axis=0)) <= 1:
#         comparison_columns.remove(column)
        continue
    else:
        stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
        comparisons.append(column)
        pvals.append(pval)
    

'''Correct for multiple testing to determine if each comparison meets the new cutoff'''
results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=.05, method='fdr_bh')
reject = results[0]


'''Format results in a pandas dataframe'''
results_df = pd.DataFrame(columns=['Comparison','P_Value'])

for i in range(0, len(reject)):
    if reject[i]:
        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


'''Sort dataframe by ascending p-value'''
results_df = results_df.sort_values(by='P_Value', ascending=True)
results_df = results_df.reset_index(drop=True)

In [64]:
def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, correction_method='bonferroni'):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)

        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)

        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
        
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
        for column in comparison_columns:
            if len(partition1[column].dropna(axis=0)) <= 1:
        #         comparison_columns.remove(column)
                continue
            elif len(partition2[column].dropna(axis=0)) <= 1:
        #         comparison_columns.remove(column)
                continue
            else:
                stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
                comparisons.append(column)
                pvals.append(pval)
            
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            results_df['Comparison'] = comparisons
            results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            return None


    except:
        print("Incorrectly Formatted Dataframe!")
        return None

In [68]:
wrap_ttest(prot_and_mutations, "Mutation", correction_method="fdr_bh", alpha=.9)

In [69]:
wrap_ttest(prot_and_mutations, "Mutation", alpha=.9)

In [34]:
wrap_ttest(prot_and_mutations, "Mutation", return_all=True) 

,Comparison,P_Value
0,SLC25A14,0.000645
1,CORO7-PAM16,0.000877
2,RIPK1,0.001035
3,LYRM9,0.001181
4,GAS6,0.001208
...,...,...
10664,MYOC,0.999707
10665,STK35,0.999726
10666,GSTT2B,0.999833
10667,CA4,0.999840
